# State of art libraries for Approximate nearest neighbor search for my favorite LightFM data set. 
The LightFM's built-in Dataset class to build an interaction dataset from raw data.  https://archive.org/details/stackexchange

The goal is to demonstrate how to use different ANN Search works 
* LSH
* Exhaustive search
* Product quantization
* Trees and graphs
* HNSW

Data Set Dependencies:
* Annoy 
* NMSLIB


In [60]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
import time
!pip install lightFM
from lightfm.datasets import fetch_stackexchange

#movielens = fetch_movielens()
data = fetch_stackexchange('crossvalidated',
                           test_set_fraction=0.1,
                           indicator_features=False,
                           tag_features=True)

In [61]:
for key, value in data.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (3213, 72360)
test <class 'scipy.sparse.coo.coo_matrix'> (3213, 72360)
item_features <class 'scipy.sparse.csr.csr_matrix'> (72360, 1246)
item_feature_labels <class 'numpy.ndarray'> (1246,)


In [62]:
stackexchange = fetch_stackexchange('crossvalidated',indicator_features=True,tag_features=True)
train = data['train']
test = data['test']

In [63]:
from lightfm import LightFM

# Set the number of threads; you can increase this
# ify you have more physical cores available.
NUM_THREADS = 2
NUM_COMPONENTS = 30
NUM_EPOCHS = 3
ITEM_ALPHA = 1e-6

# Let's fit a WARP model: these generally have the best performance.
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)

# Run 3 epochs and time it.
%time model = model.fit(train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

CPU times: user 710 ms, sys: 6.78 ms, total: 717 ms
Wall time: 392 ms


As a means of sanity checking, let's calculate the model's AUC on the training set first. If it's reasonably high, we can be sure that the model is not doing anything stupid and is fitting the training data well.

In [64]:
# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model, train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Collaborative filtering train AUC: 0.8891438


Fantastic, the model is fitting the training set well. But what about the test set..

In [65]:
# We pass in the train interactions to exclude them from predictions.
test_auc = auc_score(model, test, train_interactions=train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)

Collaborative filtering test AUC: 0.41064462


 Re-evaluating the model.



In [66]:
# Set biases to zero
model.item_biases *= 0.0

test_auc = auc_score(model, test, train_interactions=train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)

Collaborative filtering test AUC: 0.5331501


In [67]:
!pip install nmslib
import nmslib

# initialize a new nmslib index, using a HNSW index on Cosine Similarity
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [68]:
model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

# Print the Dataset stackexchange

In [69]:
print(stackexchange['item_features'])

  (0, 0)	1.0
  (0, 72360)	1.0
  (0, 72361)	1.0
  (0, 72362)	1.0
  (1, 1)	1.0
  (1, 72363)	1.0
  (1, 72364)	1.0
  (2, 2)	1.0
  (2, 72365)	1.0
  (2, 72366)	1.0
  (3, 3)	1.0
  (3, 72363)	1.0
  (3, 72367)	1.0
  (4, 4)	1.0
  (4, 72368)	1.0
  (5, 5)	1.0
  (5, 72369)	1.0
  (5, 72370)	1.0
  (5, 72371)	1.0
  (5, 72372)	1.0
  (6, 6)	1.0
  (6, 72373)	1.0
  (7, 7)	1.0
  (7, 72374)	1.0
  (7, 72375)	1.0
  :	:
  (72354, 72837)	1.0
  (72354, 73124)	1.0
  (72354, 73164)	1.0
  (72355, 72355)	1.0
  (72355, 72436)	1.0
  (72355, 72548)	1.0
  (72355, 73090)	1.0
  (72356, 72356)	1.0
  (72356, 72440)	1.0
  (72356, 72513)	1.0
  (72356, 72731)	1.0
  (72356, 72796)	1.0
  (72356, 73057)	1.0
  (72357, 72357)	1.0
  (72357, 72404)	1.0
  (72357, 73399)	1.0
  (72358, 72358)	1.0
  (72358, 72406)	1.0
  (72358, 72411)	1.0
  (72358, 72747)	1.0
  (72358, 72920)	1.0
  (72359, 72359)	1.0
  (72359, 72503)	1.0
  (72359, 72507)	1.0
  (72359, 72616)	1.0


Load pretrained models from a .pkl file

In [97]:
import pickle
with open('stackexchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors,"features":stackexchange['item_features']}, f)
print(stackexchange['item_feature_labels'])    


['question_id:0' 'question_id:1' 'question_id:2' ... 'events'
 'mutlivariate' 'sample-variance']


In [98]:
import pickle
def load_data():
    with open('stackexchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data
data = load_data()

Approximate based on the word2vec model for the dataset

In [99]:
item_features = stackexchange['item_features']
tag_labels = stackexchange['item_feature_labels']

print('There are %s distinct tags, with values like %s.' % (item_features.shape[1], tag_labels[:3].tolist()))
# Define a new model instance
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
                no_components=NUM_COMPONENTS)

# Fit the hybrid model. Note that this time, we pass
# in the item features matrix.
model = model.fit(train,
                item_features=item_features,
                epochs=NUM_EPOCHS,
                num_threads=NUM_THREADS)

There are 73606 distinct tags, with values like ['question_id:0', 'question_id:1', 'question_id:2'].


In [100]:
tag_labels = stackexchange['item_feature_labels']
def get_similar_tags(model, tag_id):
    # Define similarity as the cosine of the angle
    # between the tag latent vectors
    
    # Normalize the vectors to unit length
    tag_embeddings = (model.item_embeddings.T
                      / np.linalg.norm(model.item_embeddings, axis=1)).T
    
    query_embedding = tag_embeddings[tag_id]
    similarity = np.dot(tag_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:4]
    
    return most_similar


for tag in (u'bayesian', u'regression', u'survival'):
    tag_id = tag_labels.tolist().index(tag)
    print('Most similar tags for %s: %s' % (tag_labels[tag_id],
                                            tag_labels[get_similar_tags(model, tag_id)]))

Most similar tags for bayesian: ['prior' 'question_id:23252' 'question_id:41988']
Most similar tags for regression: ['question_id:38606' 'question_id:48040' 'question_id:20061']
Most similar tags for survival: ['cox-model' 'question_id:15719' 'question_id:63613']


Exhaustive search

In [107]:
import scipy.sparse
import random
import itertools
cx = scipy.sparse.coo_matrix(stackexchange['item_features'])
def using_coo(index):
    arr=[]
       
    for i,j,v in zip(cx.row, cx.col, cx.data):
      if i==index and i != j:
        arr.append(data['name'][j])
      elif i>index:
        break
    return arr
      # print(j,data['name'][j])
using_coo(0)

['bayesian', 'prior', 'elicitation']

In [108]:
class exSearch():
    def __init__(self, vectors, labels):
        self.vectors = vectors.astype('float32')
        self.labels = labels
        self.index = faiss.IndexFlatL2(vectors.shape[1])
        self.index.add(self.vectors)
    def get_similar_tagged_questions(self,vectors,k=12):
        distances, indices = self.index.search(vectors, k) 
        similar_q_w_feature={}
        q_features=[]
        for i in indices[0]:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature
       

In [109]:
# Exhaustive search for brute-force approach to combinatorial finding similar item in dataset 

In [110]:
!pip install faiss
!apt install libomp-dev
!python3 -m pip install --upgrade faiss-gpu==1.7.1
import faiss
index = exSearch(data["vector"], data["name"])
index.get_similar_tagged_questions(data['vector'][0:4])

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:15699': ['bayesian', 'prior', 'fitting'],
 'question_id:18112': ['bayesian'],
 'question_id:2001': ['bayesian', 'prior'],
 'question_id:2490': ['bayesian', 'prior', 'distributions'],
 'question_id:29552': ['bayesian', 'prior', 'elicitation'],
 'question_id:31509': ['bayesian', 'distributions'],
 'question_id:31875': ['bayesian', 'prior'],
 'question_id:45542': ['bayesian', 'prior', 'elicitation'],
 'question_id:4611': ['bayesian', 'maximum-likelihood'],
 'question_id:8661': ['bayesian', 'prior'],
 'question_id:9524': ['bayesian', 'prior']}

LSH

In [112]:
#LSH (Locality sensitive Hashing), an approach that enable us to search for approximate nearest neighbors for a given point in metric space
class LSHashing():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def lsh_build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, self.dimention, number_of_partition, search_in_x_partitions, subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)

    def get_similar_tagged_questions(self, vectors, k=12):
        distances, indices = self.index.search(vectors, k) 
        similar_q_w_feature={}
        q_features=[]
        for i in indices[0]:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature

In [113]:
index = LSHashing(data["vector"], data["name"])
index.lsh_build()
index.get_similar_tagged_questions(data["vector"][0:2])

{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:19616': ['bayesian',
  'machine-learning',
  'nonparametric',
  'stochastic-processes',
  'nonparametric-bayes'],
 'question_id:29362': ['bayesian',
  'standard-deviation',
  'r',
  'estimation',
  'robust'],
 'question_id:35465': ['bayesian',
  'summary-statistics',
  'confidence-interval',
  'posterior'],
 'question_id:36333': ['bayesian',
  'machine-learning',
  'regression',
  'logistic',
  'bayes'],
 'question_id:36871': ['bayesian',
  'references',
  'self-study',
  'nonparametric-bayes'],
 'question_id:37068': ['bayesian', 'machine-learning', 'regression'],
 'question_id:37689': ['bayesian', 'correlation', 'data-visualization'],
 'question_id:39589': ['bayesian',
  'hypothesis-testing',
  'proportion',
  'ab-test'],
 'question_id:43811': ['bayesian',
  'variance',
  'normal-distribution',
  'variability'],
 'question_id:45996': ['bayesian',
  'prior',
  'marginal',
  'conditioning',
  'prediction'],
 'question

Product quantization

In [116]:
class PQRO():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def prod_quant_build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimention, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)    
    def get_similar_tagged_questions(self, vectors, k=12):
        distances, indices = self.index.search(vectors, k) 
        similar_q_w_feature={}
        q_features=[]
        for i in indices[0]:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature

Product quantization is an effective vector quantization
approach to compactly encode high-dimensional vectors
for fast approximate nearest neighbor (ANN) search.  The
essence of product quantization is to decompose the original high-dimensional space into the Cartesian product of
a finite number of low-dimensional subspaces that are then
quantized separately. 

In [117]:
index = PQRO(data["vector"], data["name"])
index.prod_quant_build()
index.get_similar_tagged_questions(data["vector"][0:2])

{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:19616': ['bayesian',
  'machine-learning',
  'nonparametric',
  'stochastic-processes',
  'nonparametric-bayes'],
 'question_id:29362': ['bayesian',
  'standard-deviation',
  'r',
  'estimation',
  'robust'],
 'question_id:35465': ['bayesian',
  'summary-statistics',
  'confidence-interval',
  'posterior'],
 'question_id:36333': ['bayesian',
  'machine-learning',
  'regression',
  'logistic',
  'bayes'],
 'question_id:36871': ['bayesian',
  'references',
  'self-study',
  'nonparametric-bayes'],
 'question_id:37068': ['bayesian', 'machine-learning', 'regression'],
 'question_id:37689': ['bayesian', 'correlation', 'data-visualization'],
 'question_id:39589': ['bayesian',
  'hypothesis-testing',
  'proportion',
  'ab-test'],
 'question_id:43811': ['bayesian',
  'variance',
  'normal-distribution',
  'variability'],
 'question_id:45996': ['bayesian',
  'prior',
  'marginal',
  'conditioning',
  'prediction'],
 'question

HNSW (Hierarchical Navigable Small Worlds)

In [118]:
class HNSW():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def hnsw_build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
    
    def get_similar_tagged_questions(self, vector, k=12):
        indices = self.index.knnQuery(vector, k=k)
        similar_q_w_feature={}
        q_features=[]
        for i in indices[0]:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature

Hierarchical Navigable Small World (HNSW) graphs are among the top-performing indexes for vector similarity search. HNSW is a hugely popular technology that time and time again produces state-of-the-art performance with super fast search speeds and fantastic recall.

In [120]:
index = HNSW(data["vector"], data["name"])
index.hnsw_build()
index.get_similar_tagged_questions(data["vector"][2])

{'question_id:151': ['open-source', 'books'],
 'question_id:2': ['software', 'open-source'],
 'question_id:205': ['software', 'modeling', 'real-time'],
 'question_id:340': ['data-mining'],
 'question_id:447': ['open-source'],
 'question_id:46': ['r'],
 'question_id:62': ['data-mining'],
 'question_id:64': ['open-source', 'data-visualization'],
 'question_id:729': ['data-visualization'],
 'question_id:762': ['teaching'],
 'question_id:7931': ['software', 'open-source', 'teaching'],
 'question_id:948': ['software']}

Trees and graphs

In [121]:
class TG():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def tree_build(self, number_of_trees=4):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
  
    def get_similar_tagged_questions(self, vector, k=12):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        similar_q_w_feature={}
        q_features=[]
        for i in indices:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature
       

Propose an index structure, the ANN-tree/graphs (approximate nearest neighbor tree) to solve this problem. The ANN-tree supports high accuracy nearest neighbor search. The actual nearest neighbor of a query point can usually be found in the first leaf page accessed. 



In [124]:
import annoy
index = TG(data["vector"], data["name"])
index.tree_build()
index.get_similar_tagged_questions(data["vector"][0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:14139': ['bayesian', 'hypothesis-testing', 'self-study'],
 'question_id:20253': ['bayesian'],
 'question_id:20474': ['bayesian'],
 'question_id:34221': ['bayesian',
  'prior',
  'distributions',
  'central-limit-theorem'],
 'question_id:36078': ['bayesian'],
 'question_id:36520': ['bayesian'],
 'question_id:36713': ['bayesian', 'frequentist'],
 'question_id:41860': ['bayesian'],
 'question_id:53430': ['bayesian', 'prior'],
 'question_id:60949': ['bayesian', 'prior'],
 'question_id:69684': ['bayesian']}

# Summary
Based on above  HNSW(NMSLib) is by far and away the best choice here. Trees and Graphs’s(annoy) performance is the worst at this particular task, it performs much better with cosine based lookup (like when computing similar items). Also, the indices are all memory mapped from file, which makes it much more suitable if you have multiple python processes serving up requests.


